In [ ]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))

    return epochs, pp_directory

In [ ]:
# def rotate_trials(task, pp, epochs):

#     if pp % 2 == 0:
#         if task == 'rot':
#             #get trial after first two tasks (trial 96 to 185)
#             epochs = epochs[96:186]
#         elif task == 'rdm':
#             #get trial after first task (trial 48 to 95)
#             epochs = epochs[48:96]
#     elif pp % 2 == 1:
#         if task == 'rot':
#             #get trial after first five tasks (trial 282 to 371)
#             epochs = epochs[282:372]
#         elif task == 'rdm':
#             #get trial after first five tasks (trial 234 to 281)
#             epochs = epochs[234:282]
    
#     return(epochs)

In [ ]:
#get index of trials from behavioral data
def epoch_index(task, listval, pp):
    if task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ROT_learningcurve_degrees.csv')
    elif task == 'rdm':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\RDMROT_learningcurve_degrees.csv')
    elif task == 'aln':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ALIGNED_learningcurve_degrees.csv')
        
    ppdata = list(df.iloc[:,pp+1])
    
    index = [i for i,v in enumerate(ppdata) if listval[0] < v < listval[1] or listval[2] < v < listval[3]]
    
    #epochs_out = epochs[index]
    numtrials = len(index)
    
    #return epochs_out, numtrials
    return index, numtrials

In [ ]:
def direction_index(direction, pp):
    
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ALIGNED_MovementWorkspace_direction.csv')
    
    ppdata = list(df.iloc[:,pp+1])

    index = [i for i,v in enumerate(ppdata) if direction == v]

    numtrials = len(index)
    
    return index, numtrials

In [ ]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]

        elif task == 'rdm':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'rdm':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [ ]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, condname, task):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s-epo.fif' % (pp_num, condname, task))
    data.save(out_fname, overwrite = True)

In [ ]:
#save as an evoked object
def output_condition_evoked(pp_num, data, ppdir, condname, task):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s-ave.fif' % (pp_num, condname, task))
    evoked.save(out_fname, overwrite = True)

In [ ]:
#save as evoked objects
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30
#within X degrees is considered a "hit"
err_cutoff = [-3, 0, 0, 3]


#aligned
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx, numtrials = epoch_index('aln', err_cutoff, pp)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index(conditions[cond], pp)
        condidx = [i for i in idx if i in diridx]
        aldata = epoch_trials('aln', pp, data, condidx)
        output_condition_epochs(pp, aldata, ppdir, conditionnames[cond], 'aligned')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, aldata, ppdir, conditionnames[cond], 'aligned')


In [ ]:
#get grand averages - split according to error conditions
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']

for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in participants:
        id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], conditionnames[condition], 'aligned'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
        if condition == 0:
            grand_right = mne.grand_average(evoked_list)
            right_flist = evoked_list
        elif condition == 1:
            grand_left = mne.grand_average(evoked_list)
            left_flist = evoked_list



In [ ]:
#runs for LRP ONLY: need separate files for C3 and C4

flists = ['right', 'left']
channels = ['C3', 'C4']
for channel in channels:
    for f in flists:
        newdf = pd.DataFrame()
        for pp in participants:
            if f == 'right':
                df = right_flist[pp].to_data_frame()
            elif f == 'left':
                df = left_flist[pp].to_data_frame()
             
            time = df['time'].tolist()
            df = df[[channel]]
            ppdat = []
            for timept in range(0,300):
                subdf = df.loc[timept,:].mean()
                ppdat.append(subdf)
    
            newdf['pp%03d'% pp] = ppdat
    
        newdf['time'] = time
        newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s_%s.csv' % (f, erps, channel))
        newdf.to_csv(newdf_filename)

In [ ]:
# Difference waves between aligned and perturbed conditions for C3
#channels = ['C3']
channels = ['C4']
# channels = ['Oz','O1','O2']
evokeds = dict(right = right_flist, 
               left = left_flist)
#plot
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)